In [1]:
import requests 
import pandas as pd
import json

import sys
sys.path.append("../") 

from constants import ACCOUNT_ID, API_KEY, OANDA_URL

from typing import List, Dict, AnyStr, Optional


In [2]:
# create a new session
session = requests.Session()

session_headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
} 

# this makes sure that all requests are made with the above headers, as required by Oanda API
session.headers.update(session_headers)

In [3]:
def build_url(paths: List | None, params: Dict = {}) -> str:
    url = OANDA_URL
    for path in paths:
        url += f"/{path}"

    for param, value in params.items():
        url += f"?{param}={value}"
        url += "&"

    return url

In [4]:
build_url(paths=['accounts', ACCOUNT_ID, 'pricing'], params={'instruments': 'EUR_USD'})

'https://api-fxpractice.oanda.com/v3/accounts/101-004-29541766-001/pricing?instruments=EUR_USD&'

# 1. Fetch Trading Instruments (Pairs)

In [7]:
params = dict(
    count=10,granularity='H1',
)

url = build_url(['accounts', ACCOUNT_ID, 'instruments'])

response = session.get(url=url, params=None, headers=None)
data = response.json()

In [36]:
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [8]:
response.status_code

200

In [9]:
instruments_list = data['instruments']
len(instruments_list)

123

In [11]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [17]:
keys_of_interest = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

instruments_obj = {}


for i in instruments_list:
    key = i['name']
    instruments_obj[key] = { k: i[k] for k in keys_of_interest}

instruments_obj['USD_ZAR']

{'name': 'USD_ZAR',
 'type': 'CURRENCY',
 'displayName': 'USD/ZAR',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.05'}

In [18]:
# save the instruments as json file
with open("../data/instruments.json", 'w') as f:
    f.write(json.dumps(instruments_obj, indent=2))

print("instruments.json saved successfully!")

instruments.json saved successfully!


# 2. Fetch Candles (Historical Data)

In [20]:
def fetch_candles(pair_name: str, count: int = 10, granularity: str = 'H1') -> tuple[int, list]:
    url = build_url(paths=['instruments', pair_name, 'candles'])
    params = dict(
        count=count, 
        granularity=granularity,
        price="MBA"                 # mid, bid and ask prices
    )
    response = session.get(url=url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' in data:
            data = data['candles']
        else: 
            data = []

    return response.status_code, data
    

In [21]:
code, data = fetch_candles('USD_ZAR')
code

200

In [22]:
len(data)

10

In [47]:
def get_candles_df(data: list) -> pd.DataFrame:

    if len(data) == 0: return pd.DataFrame()       # in case data is empty

    final_data = []

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    for candle in data:
        if not candle['complete']: continue 
        new_dict = {}
        new_dict['time'] = candle['time']

        for p in prices:
            for o in ohlc:
                new_dict[f'{p}_{o}'] = float(candle[p][o])

        new_dict['volume'] = candle['volume']
        final_data.append(new_dict)

    df = pd.DataFrame(final_data)
    df['time'] = pd.to_datetime(df['time'])
    return df

In [43]:
get_candles_df(fetch_candles('EUR_USD', 100, 'H4')[1])

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,volume
0,2025-09-11 05:00:00+00:00,1.16996,1.16996,1.16838,1.16921,1.16988,1.16988,1.16830,1.16913,1.17004,1.17004,1.16846,1.16929,25427
1,2025-09-11 09:00:00+00:00,1.16920,1.17210,1.16606,1.17152,1.16912,1.17202,1.16587,1.17145,1.16929,1.17218,1.16618,1.17160,47243
2,2025-09-11 13:00:00+00:00,1.17152,1.17461,1.17140,1.17365,1.17144,1.17453,1.17131,1.17357,1.17160,1.17469,1.17148,1.17373,50660
3,2025-09-11 17:00:00+00:00,1.17366,1.17398,1.17278,1.17355,1.17358,1.17390,1.17269,1.17346,1.17373,1.17405,1.17286,1.17364,14890
4,2025-09-11 21:00:00+00:00,1.17333,1.17406,1.17273,1.17330,1.17306,1.17397,1.17264,1.17321,1.17360,1.17417,1.17281,1.17338,8924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-10-03 01:00:00+00:00,1.17243,1.17293,1.17162,1.17180,1.17235,1.17285,1.17154,1.17172,1.17251,1.17301,1.17171,1.17188,13591
96,2025-10-03 05:00:00+00:00,1.17180,1.17446,1.17149,1.17402,1.17171,1.17439,1.17140,1.17395,1.17188,1.17454,1.17158,1.17410,24354
97,2025-10-03 09:00:00+00:00,1.17402,1.17429,1.17270,1.17395,1.17394,1.17422,1.17259,1.17387,1.17410,1.17437,1.17279,1.17403,25225
98,2025-10-03 13:00:00+00:00,1.17396,1.17594,1.17327,1.17398,1.17387,1.17586,1.17320,1.17391,1.17404,1.17602,1.17334,1.17406,43076


In [48]:
get_candles_df([])

""


In [ ]:
def create_data_file(pair_name: str, count: int = 10, granularity: str = 'H1'):
    status_code, data = fetch_candles(pair_name=pair_name, count=count, granularity=granularity)
    
    if status_code != 200:
        print("Something went wrong while fetching data!")
        print(f"Status code: {status_code}\n Instrument: {pair_name}")
        print(data)
        return
    
    if len(data) == 0:
        print("Request successfull, but no data was returned!")
        print(f"Status code: {status_code}\n Instrument: {pair_name}")
        return
    
    df = get_candles_df(data=data)
    min_time = df.time.dt.date.min()
    max_time = df.time.dt.date.max()
    df_filename = f"{pair_name}_{granularity}_{min_time}_{max_time}.csv"
    df.to_csv(f"../data/history/{df_filename}", index=False)
    print(f"{df_filename} saved! || {df.shape[0]} {granularity} candles  ||  {min_time}  -  {max_time}")

    

In [91]:
create_data_file('EUR_USD')

EUR_USD_H1_2025-10-03_2025-10-03.csv saved! // 10 H1 candles  //  2025-10-03  -  2025-10-03


In [101]:
currencies = ['EUR', 'USD', 'NZD', 'ZAR', 'JPY', 'CAD', 'GBP', 'AUD']
granularities = ['H1', 'H4']

for p1 in currencies:
    for p2 in currencies:
        if p1 == p2: continue
        pair = f"{p1}_{p2}"
        if pair in instruments_obj.keys():
            for granularity in granularities:
                create_data_file(pair_name=pair, count=4001, granularity=granularity)

EUR_USD_H1_2025-02-13_2025-10-03.csv saved! // 4001 H1 candles  //  2025-02-13  -  2025-10-03
EUR_USD_H4_2023-03-10_2025-10-03.csv saved! // 4001 H4 candles  //  2023-03-10  -  2025-10-03
EUR_NZD_H1_2025-02-13_2025-10-03.csv saved! // 4001 H1 candles  //  2025-02-13  -  2025-10-03
EUR_NZD_H4_2023-03-10_2025-10-03.csv saved! // 4001 H4 candles  //  2023-03-10  -  2025-10-03
EUR_ZAR_H1_2025-02-13_2025-10-03.csv saved! // 4001 H1 candles  //  2025-02-13  -  2025-10-03
EUR_ZAR_H4_2023-03-10_2025-10-03.csv saved! // 4001 H4 candles  //  2023-03-10  -  2025-10-03
EUR_JPY_H1_2025-02-13_2025-10-03.csv saved! // 4001 H1 candles  //  2025-02-13  -  2025-10-03
EUR_JPY_H4_2023-03-10_2025-10-03.csv saved! // 4001 H4 candles  //  2023-03-10  -  2025-10-03
EUR_CAD_H1_2025-02-13_2025-10-03.csv saved! // 4001 H1 candles  //  2025-02-13  -  2025-10-03
EUR_CAD_H4_2023-03-10_2025-10-03.csv saved! // 4001 H4 candles  //  2023-03-10  -  2025-10-03
EUR_GBP_H1_2025-02-13_2025-10-03.csv saved! // 4001 H1 candl

In [100]:
instruments_obj

{'TRY_JPY': {'name': 'TRY_JPY',
  'type': 'CURRENCY',
  'displayName': 'TRY/JPY',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.25'},
 'AUD_JPY': {'name': 'AUD_JPY',
  'type': 'CURRENCY',
  'displayName': 'AUD/JPY',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'USB02Y_USD': {'name': 'USB02Y_USD',
  'type': 'CFD',
  'displayName': 'US 2Y T-Note',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.20'},
 'XAU_USD': {'name': 'XAU_USD',
  'type': 'METAL',
  'displayName': 'Gold',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'USD_CNH': {'name': 'USD_CNH',
  'type': 'CURRENCY',
  'displayName': 'USD/CNH',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'NZD_JPY': {'name': 'NZD_JPY',
  'type': 'CURRENCY',
  'displayName': 'NZD/JPY',
  'pipLocatio

In [3]:
today=pd.to_datetime("2025-10-10")

In [7]:
import datetime 

min(today + datetime.timedelta(seconds=240), today)

Timestamp('2025-10-10 00:00:00')

In [8]:
pd.to_datetime(today)

Timestamp('2025-10-10 00:00:00')

import date